In [ ]:
from multiprocessing import Pool, Lock
import os

from datetime import datetime

from mortalityForecast import mortalityForecast as mf
from mortalityForecast import mortality_data_loader as data_loader
from mortalityForecast import utils

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import product
from itertools import cycle
import psutil 

In [ ]:
first_year = 1951
train_length = 60
forecast_length = 10
last_year = 1999

param_tuple = tuple()

start_years = range(first_year, last_year - train_length - forecast_length + 2)
countries = [data_loader.SWEDEN]
nn_layers = [1]
latent_dims = [1,2,3,4,5,6,7,8,9]

In [ ]:
for start_year, country, nn_layer, latent_dim in product(start_years, countries, nn_layers, latent_dims):
    param_tuple = param_tuple + ({
    'country' : country,
    'first_year_train' : start_year,
    'last_year_train' : start_year + train_length - 1,
    'first_year_test' : start_year + train_length,
    'last_year_test' : start_year + train_length + forecast_length-1,
    'sex' : "Male", # or Female, Both
    'max_age' : 100,
    'nn_layers' : nn_layer,
    'latent_dim' : latent_dim},)

for idx, param in enumerate(param_tuple):
    param['id'] = idx +1    

l = Lock()

In [ ]:
def evaluate_model_and_save(param):

    

    log_score = model.evaluate(exposure_test, deaths_test, mc_samples = 10000)

    data = list(
    zip(
        cycle(['sweden']),
        cycle([param['first_year_train']]),
        cycle([param['last_year_train']]),
        cycle([param['sex']]),
        cycle([param['max_age']]),
        cycle([param['nn_layers']]),
        cycle([param['latent_dim']]),
        range(1,len(log_score)+1),
        log_score,
        cycle([datetime.now().strftime("%d/%m/%Y")]),
        cycle([datetime.now().strftime("%H:%M:%S")])
        )
    )
    
    df = pd.DataFrame(data,
                  columns = ['Country', 'first_year_train', 'last_year_train', 'Sex','max_age', 'nn_layers', 'latent_dim', 'Forecast horizon', 'Log score', 'Date', 'Time'])


    out_path = 'results-230126' + '.csv'
    
    l.acquire()
    try:
        df.to_csv(out_path, mode='a', header=not os.path.exists(out_path), index = False)
        print(param['id'])
    finally:
        l.release() """